# DSCI 100 Group 10: Project 


# **TO DO LIST FOR INTRODUCTION**
## - Background information on the topic so people can understand the rest of the report
## - **Clearly** state the question we are trying to answer
## - Identify and describe the dataset used to answer the question

# **TO DO LIST FOR METHODS**
## - Load the data from the web (pref from the original source, github if not)
## - Tidy, clean and wrangle the data
## - Summary of the data for exploratory reasons
## - Figures of the data for exploratory reasons
## - Data analysis (throughout, have plots)
#### - Splitting the data into training and testing
#### - Creating the kknn algorithm using the training dataset (using k-folds as well)
#### - Testing the algorithm using the testing set
#### - A analysis of accuracy (metrics)
## - A final summary plot

# **TO DO LIST FOR DISCUSSION**
## - Summary
## - Expectations (hypothesis), impact of the finding (application to irl) and future question


# **THINGS TO KEEP IN MIND**
#### - Use set.seed()
#### - Lack of explanation for the predictor variables
#### - Visuals do not have titles or labels
#### - All figures and tables should have table/figure number and legend (when applicable)
#### - Our reasoning is off, we need to improve our reasoning